# TP7 - Connexion à MongoDB sous `Python`

#### Analyse de Données Massives - Master 1ère année

On utilise dans ce *notebook* les modules `pymongo`, `pandas` et `pprint`.

In [ ]:
import pymongo
import pandas
import pprint

On créé ensuite la connexion vers *MongoDB*. Et on vérifie que la base `test` contient bien la collection `restaurants`.

In [46]:
con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")
con.test.collection_names()

['restaurants']

On va aussi utiliser ici la fonction `affiche()` créée dans le TP pour améliorer la lisibilité de l'affichage des résultats.

In [1]:
def affiche(res):
    for r in res:
        pprint.pprint(r)

## Exercices

Nous allons utiliser la base de données `test`, dans laquelle se trouve la collection `restaurants`. Celle-ci est composée de plus de 25000 retaurants new-yorkais, avec des évaluations de ceux-ci.

### 1. Créer une connexion vers cette collection

On put simplfier l'écriture des requêtes à suivre en faisant la connexion directement avec la collection `restaurants`.

In [2]:
db = con.test.restaurants

### 2. Déterminer le nombre exact de restaurants.

Un simple dénombrement est à faire ici.

In [3]:
db.count()

25359

### 3. Lister les différents quartiers représentés (*borough*) ainsi que les différents types de  cuisine (*cuisine*).

On utilse la fonction `distinct()` ici pour lister les différentes valeurs de `borough`. On remarque que pour certains restaurants le quartier n'est pas connu.

In [4]:
db.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

Idem pour lister les (nombreux) différents types de cuisine. On stocke le résultat dans une variable pour dénombrer les styles (`85`). Et nous n'affichons que les 10 premiers pour exemple.

In [49]:
type_cuisine = db.distinct("cuisine")
len(type_cuisine)

85

In [50]:
type_cuisine[:10]

['Bakery',
 'Hamburgers',
 'Irish',
 'American',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Turkish']

### 4. Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (*cuisine* égale à *Soups*).

C'est la fonction `find()` qui nous permet de récupérer ces informations (4 restaurants).

In [6]:
res = db.find({ "cuisine": "Soups" }, { "_id": 0, "name": 1, "borough": 1})
affiche(res)

{'borough': 'Manhattan', 'name': 'Wichcraft Express'}
{'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'}
{'borough': 'Manhattan', 'name': 'The Original Soupman'}
{'borough': 'Manhattan', 'name': 'Peasant Stock'}


### 5. Donner le nombre de restaurants ayant eu au moins une fois le *grade* (dans *grades*) `Z`.

Pour cela, nous pouvons déjà nous retreindre aux restaurants ayant eu le *grade* `Z`, puis nous comptons la taille du résultat avec `count()`.

In [7]:
db.find({ "grades.grade": "Z"}).count()

1337

### 6. Calculer le nombre de restaurants pour chaque quartier

Ici, un aggrégat basique est à faire, avec `aggregate()`. La plupart sont dans *Manhattan*, mais on remarque aussi qu'on a 51 restaurants sans quartier connu.

In [8]:
res = db.aggregate([ { "$group": { "_id": "$borough", "nb": { "$sum": 1 } } } ])
affiche(res)

{'_id': 'Missing', 'nb': 51}
{'_id': 'Staten Island', 'nb': 969}
{'_id': 'Brooklyn', 'nb': 6086}
{'_id': 'Bronx', 'nb': 2338}
{'_id': 'Queens', 'nb': 5656}
{'_id': 'Manhattan', 'nb': 10259}


### 7. Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)

En plus d'un calcul d'aggrégat avec `$group`, on tri le résultat par ordre décroissant sur le nombre et on se limite aux 5 premières valeurs avec `$limit`.

In [51]:
res = db.aggregate([ 
    { "$group": { "_id": "$cuisine" , "nb" : { "$sum" : 1 } } },
    { "$sort" : { "nb": -1 } },
    { "$limit" : 5 }
])
affiche(res)

{'_id': 'American', 'nb': 6183}
{'_id': 'Chinese', 'nb': 2418}
{'_id': 'Café/Coffee/Tea', 'nb': 1214}
{'_id': 'Pizza', 'nb': 1163}
{'_id': 'Italian', 'nb': 1069}


### 8. Pour chaque *grade* possible, donner le *score* moyen, minimum et maximum.

Il faut d'abord dégrouper les évaluations pour chaque restaurant grâce à `$unwind`. Ensuite, c'est un calcul assez simple à réaliser.

In [10]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": "$grades.grade", 
        "nb" : { "$sum" : 1 },
        "moy" : { "$avg" : "$grades.score" },
        "min" : { "$min" : "$grades.score" },
        "max" : { "$max" : "$grades.score" }
    } }
])
affiche(res)

{'_id': 'P', 'max': 43, 'min': 0, 'moy': 5.506265664160401, 'nb': 1197}
{'_id': 'A', 'max': 27, 'min': 0, 'moy': 9.021954029147022, 'nb': 74656}
{'_id': 'Not Yet Graded', 'max': 75, 'min': 0, 'moy': 15.853515625, 'nb': 525}
{'_id': 'B', 'max': 57, 'min': -1, 'moy': 20.002459731809886, 'nb': 12603}
{'_id': 'Z', 'max': 71, 'min': -1, 'moy': 23.702318623784592, 'nb': 1337}
{'_id': 'C', 'max': 131, 'min': -1, 'moy': 30.70079491255962, 'nb': 3145}


In [55]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": "$grades.grade", 
        "nb" : { "$sum" : 1 },
        "moy" : { "$avg" : "$grades.score" },
        "min" : { "$min" : "$grades.score" },
        "max" : { "$max" : "$grades.score" }
    } },
    { "$project" : {
        "moyr" : 'Math.round("$moy")'
    }}
])
affiche(res)

{'_id': 'P', 'moyr': 'Math.round("$moy")'}
{'_id': 'A', 'moyr': 'Math.round("$moy")'}
{'_id': 'Not Yet Graded', 'moyr': 'Math.round("$moy")'}
{'_id': 'B', 'moyr': 'Math.round("$moy")'}
{'_id': 'Z', 'moyr': 'Math.round("$moy")'}
{'_id': 'C', 'moyr': 'Math.round("$moy")'}


### 9. Créer un `DataFrame` contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque *grade* possible. Si un restaurant n'a pas eu d'évaluation d'un *grade*, nous aurons une valeur absente.

Si nous regardons le nombre de restaurants par nom de ceux-ci, nous remarquons qu'il y a des restaurants avec le même nom (des chaînes comme *Starbucks* ou *Subway* par exemple).

In [11]:
res = db.aggregate([
    { "$group": { "_id": "$name", "nb": { "$sum": 1 } } },
    { "$sort": { "nb": -1 } },
    { "$limit": 10 }
])
affiche(res)

{'_id': 'Subway', 'nb': 421}
{'_id': 'Starbucks Coffee', 'nb': 223}
{'_id': "Mcdonald'S", 'nb': 208}
{'_id': "Dunkin' Donuts", 'nb': 206}
{'_id': 'Dunkin Donuts', 'nb': 200}
{'_id': '', 'nb': 151}
{'_id': 'Kennedy Fried Chicken', 'nb': 88}
{'_id': 'Crown Fried Chicken', 'nb': 81}
{'_id': 'Burger King', 'nb': 80}
{'_id': "Domino'S Pizza", 'nb': 67}


Nous allons donc faire un regroupement par identifiant de restaurant (`restaurant_id`) et par `grade`, pour calculer le score moyen. Nous voyons ici les premiers résultats de l'aggrégation.

In [52]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": { "restaurant_id": "$restaurant_id", "grade": "$grades.grade" }, 
        "moy" : { "$avg" : "$grades.score" }
    } }
])
l = list(res)
l[:10]

[{'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018661'}, 'moy': 2.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018622'},
  'moy': 19.0},
 {'_id': {'grade': 'A', 'restaurant_id': '50018608'}, 'moy': 12.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018587'}, 'moy': 2.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018565'}, 'moy': 0.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018547'},
  'moy': 29.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018535'},
  'moy': 47.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018514'}, 'moy': 9.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018480'},
  'moy': 43.0},
 {'_id': {'grade': 'Not Yet Graded', 'restaurant_id': '50018450'}, 'moy': 0.0}]

Ensuite, on créé un `DataFrame` *à la main*, en récupérant la liste des identifiants, la liste des grades et la liste des scores moyens. Ce qui nous donne à la fin le `DataFrame` suivant.

In [32]:
df = pandas.DataFrame({
    "restaurant_id"  : [r["_id"]["restaurant_id"] for r in l],
    "grade": [r["_id"]["grade"] for r in l],
    "score": [r["moy"] for r in l]
})
df.head()

,grade,restaurant_id,score
0,Not Yet Graded,50018661,2.0
1,Not Yet Graded,50018622,19.0
2,A,50018608,12.0
3,Not Yet Graded,50018587,2.0
4,Not Yet Graded,50018565,0.0


Il faut maintenant transformer ce `DataFrame` en faisant l'*inverse* de la fonction `melt()`. Pour cela, nous disposons de la fonction `pivot()` à partir du `DataFrame`. Cette fonction prend trois paramètre :

- `index` : identifiant de chaque ligne (ici, l'identifiant des restaurants)
- `columns` : noms des colonnes (ici, chaque grade) 
- `value` : valeurs des cellules (ici, score moyen pour un restaurant pour un grade)

In [33]:
dfbis = df.pivot(index = "restaurant_id", columns = "grade", values = "score")
dfbis.head()

grade,A,B,C,Not Yet Graded,P,Z
restaurant_id,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN
30112340,10.666667,23.0,NaN,NaN,NaN,NaN
30191841,9.250000,NaN,NaN,NaN,NaN,NaN
40356018,9.000000,NaN,NaN,NaN,NaN,NaN
40356068,13.000000,25.0,NaN,NaN,NaN,20.0


Au final, nous regardons les informations de notre `DataFrame` ainsi créé. Nous remarquons que la plupart ont eu un grade `A`, et plusieurs n'ont pas encore été évalué.

In [34]:
dfbis.shape

(24621, 6)

In [35]:
dfbis.count()

grade
A                 23440
B                  8280
C                  2708
Not Yet Graded      512
P                  1154
Z                  1337
dtype: int64

Nous récupérons ici les informations des restaurants (nom - `name` - et quartier - `borough`) pour les ajouter à notre table précédente. Ici, nous récupérons juste les informations que l'on souhaite.

In [41]:
infos = db.find({}, { "_id": 0, "restaurant_id": 1, "name": 1, "borough": 1 })
infos_df = pandas.DataFrame(list(infos))
infos_df.head()

,borough,name,restaurant_id
0,Bronx,Morris Park Bake Shop,30075445
1,Brooklyn,Wendy'S,30112340
2,Manhattan,Dj Reynolds Pub And Restaurant,30191841
3,Brooklyn,Riviera Caterer,40356018
4,Queens,Tov Kosher Kitchen,40356068


Pour réaliser une jointure, nous avons la fonction `join()` qui réalise une jointure sur les `index` des lignes par défaut. La fonction `set_index()` permet de définir cet `index` de lignes pour les informations des restaurants, avant de faire la jointure.

In [43]:
final = dfbis.join(infos_df.set_index("restaurant_id"))
final.head()

,A,B,C,Not Yet Graded,P,Z,borough,name
restaurant_id,,,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN,Bronx,Morris Park Bake Shop
30112340,10.666667,23.0,NaN,NaN,NaN,NaN,Brooklyn,Wendy'S
30191841,9.250000,NaN,NaN,NaN,NaN,NaN,Manhattan,Dj Reynolds Pub And Restaurant
40356018,9.000000,NaN,NaN,NaN,NaN,NaN,Brooklyn,Riviera Caterer
40356068,13.000000,25.0,NaN,NaN,NaN,20.0,Queens,Tov Kosher Kitchen


Avec un `DataFrame` ainsi créé, il est maintenant possible, par exemple, de calculer les scores moyens de chaque grade, pour chaque quartier (ici, dans `python`).

In [45]:
final.groupby("borough").mean()

,A,B,C,Not Yet Graded,P,Z
borough,,,,,,
Bronx,8.837432,19.789996,28.903614,14.046512,5.669065,22.376238
Brooklyn,8.941166,19.869812,28.732290,16.412587,5.353474,23.449123
Manhattan,8.935430,19.848022,31.182162,17.836478,5.680101,24.492228
Missing,8.410000,22.000000,43.500000,NaN,NaN,NaN
Queens,9.117012,19.879747,31.122008,14.223881,5.283582,23.509259
Staten Island,9.331491,19.365970,32.820000,12.848485,6.684211,19.770833
